In [24]:
!rm -rf runs videos
!rm db.sqlite3

## Environment Setup

In [14]:
!pip install tensorboardX
!pip install pyglet==1.5.1
!pip install torchsummary
!pip install optuna
!pip install optuna-dashboard

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 365.7/365.7 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 212.9/212.9 kB 6.5 MB/s eta 0:00:00
     -------------------------------------- 155.4/155.4 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 192.1/192.1 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 4.3/4.3 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 90.2/90.2 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install setuptools==65.5.1
!pip install gym==0.21.0
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
!pip install pyvirtualdisplay
# !sudo apt-get install -y xvfb

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1024, 768))
virtual_display.start()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [19]:
!nvidia-smi

Wed May  3 13:01:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1060 6GB   WDDM | 00000000:01:00.0 Off |                  N/A |
|  0%   38C    P8                8W / 200W|      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Training

In [1]:
import sys
import os

# Get the absolute path to the parent directory of gym-tetris
gym_tetris_parent_path = os.path.abspath(os.path.join('..', 'gym-tetris'))

# Append the path to the sys.path
sys.path.append(gym_tetris_parent_path)

In [2]:
import os
import random
import time
from distutils.util import strtobool

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT

from torchsummary import summary

In [ ]:
# import importlib
# importlib.reload(gym_tetris)
# importlib.reload(gym_tetris.actions)

In [11]:
class QNetwork(nn.Module):
    def __init__(self, actions_num):
        super().__init__()
        self.network =  nn.Sequential(
            nn.Conv2d(1, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(3136, 512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512, actions_num),
        )

    def forward(self, x):
        return self.network(x / 255.0)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

### Training

In [4]:
# Making an environment
def get_env(env_id, seed, capture_video, run_name, video_freq=100):
    env = gym_tetris.make(env_id)
    env = JoypadSpace(env, MOVEMENT)
    env = gym.wrappers.RecordEpisodeStatistics(env)
    if capture_video:
        env = gym.wrappers.RecordVideo(env, f"videos/{run_name}", episode_trigger=lambda ep_num: ep_num % video_freq == 0)
            
    env = gym.wrappers.ResizeObservation(env, (84, 84))
    env = gym.wrappers.GrayScaleObservation(env)
    env = gym.wrappers.FrameStack(env, 1)
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env

In [5]:
# Evaluation
def evaluate(
    model: torch.nn.Module,
    env_id: str,
    eval_episodes: int,
    run_name: str,
    seed: int,
    device: torch.device = torch.device("cpu"),
    capture_video: bool = True,
    video_frequency: int = 1
):
    env = get_env(env_id, seed, capture_video, run_name, video_frequency) 
    
    model.eval()

    obs = env.reset()
    scores = []
    while len(scores) < eval_episodes:
        q_values = model(torch.Tensor(obs).to(device))
        action = torch.argmax(q_values, dim=1).cpu().numpy()

        next_obs, _, _, info = env.step(action)

        if "episode" in info.keys():
            print(f"eval_episode={len(scores)}, score={info['score']}, episodic_return={info['episode']['r']}")
            scores += [info["score"]]

        obs = next_obs

    env.close()
    return scores

In [6]:
# Single env training without optuna - for simplicity
def train(args):
    run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
    prefix = ""
    
    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

    # TRY NOT TO MODIFY: seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device_name = "cuda" if torch.cuda.is_available() and args.cuda else "cpu"
    device_name = "mps" if torch.backends.mps.is_available() and args.mps else device_name
    device = torch.device(device_name)

    print("device_name:", device_name)

    # env setup
    env = get_env(args.env_id, args.seed, args.capture_video, run_name, args.video_frequency)
    assert isinstance(env.action_space, gym.spaces.Discrete), "only discrete action space is supported"

    q_network = QNetwork(env.action_space.n).to(device)
    optimizer = optim.Adam(q_network.parameters(), lr=args.learning_rate)
    target_network = QNetwork(env.action_space.n).to(device)
    target_network.load_state_dict(q_network.state_dict())

    summary(q_network, input_size=(1, 84, 84), batch_size=args.batch_size, device=device_name)

    rb = ReplayBuffer(
        args.buffer_size,
        env.observation_space,
        env.action_space,
        device,
    )

    start_time = time.time()

    # TODO: do exploration-exploitation decision per-piece
    # piece_cnt = 0
    # is_new_piece = False

    # TRY NOT TO MODIFY: start the game
    obs = env.reset()
    for global_step in range(args.total_timesteps):
        # ALGO LOGIC: put action logic here
        epsilon = linear_schedule(args.start_e, args.end_e, args.exploration_fraction * args.total_timesteps, global_step)
        if random.random() < epsilon:
            action = np.array([env.single_action_space.sample() for _ in range(env.num_envs)])
        else:
            q_values = q_network(torch.Tensor(obs).to(device))
            action = torch.argmax(q_values, dim=1).cpu().numpy()

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, reward, done, info = env.step(action)

        # TRY NOT TO MODIFY: record rewards for plotting purposes
        if "episode" in info.keys():
            print(f"{prefix}global_step={global_step}, episodic_return={info['episode']['r']}, score={info['score']}")
            writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
            writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)
            writer.add_scalar("charts/epsilon", epsilon, global_step)
            writer.add_scalar("charts/score", info["score"], global_step)

        # TRY NOT TO MODIFY: save data to reply buffer; handle `terminal_observation`
        real_next_obs = next_obs.copy()

        if done:
            real_next_obs = info["terminal_observation"]  # Tetris environment does not set a terminal observation.

        rb.add(obs, real_next_obs, action, reward, done, info)

        # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
        obs = next_obs

        # ALGO LOGIC: training.
        if global_step > args.learning_starts:
            if global_step % args.train_frequency == 0:
                data = rb.sample(args.batch_size)
                with torch.no_grad():
                    target_max, _ = target_network(data.next_observations).max(dim=1)
                    td_target = data.rewards.flatten() + args.gamma * target_max * (1 - data.dones.flatten())
                old_val = q_network(data.observations).gather(1, data.actions).squeeze()
                loss = F.mse_loss(td_target, old_val)

                if global_step % 100 == 0:
                    writer.add_scalar("losses/td_loss", loss, global_step)
                    writer.add_scalar("losses/q_values", old_val.mean().item(), global_step)
                    writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

                # optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # update target network
            if global_step % args.target_network_frequency == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        args.tau * q_network_param.data + (1.0 - args.tau) * target_network_param.data
                    )

            if global_step % args.backup_frequency == 0:
                model_backup_path = f"runs/{run_name}/{args.exp_name}.backup"
                torch.save(q_network.state_dict(), model_backup_path)

    if args.save_model:
        model_path = f"runs/{run_name}/{args.exp_name}.cleanrl_model"
        torch.save(q_network.state_dict(), model_path)
        print(f"{prefix}model saved to {model_path}")

        scores = evaluate(
            q_network,
            args.env_id,
            args.eval_episodes,
            run_name=f"{run_name}-eval",
            Model=QNetwork,
            seed=args.seed,
            device=device,
            capture_video=args.capture_video
        )
        
    env.close()
    writer.close()

## Main

In [9]:
# Weeklong args
class Args:
    def __init__(self):
        # Settings
        self.exp_name = "Tetris_DQN"
        self.torch_deterministic = True
        self.cuda = True
        self.mps = False
        self.capture_video = True
        self.save_model = True
        self.eval_episodes = 4
        self.video_frequency = 10

        # Hyper-Parameters
        self.env_id = "TetrisA-v0"
        self.seed = 1
        self.total_timesteps = 100000
        self.learning_rate = 1e-4
        self.buffer_size = 20000
        self.gamma = 0.99
        self.tau = 1.0
        self.target_network_frequency = 1000
        self.batch_size = 32
        self.start_e = 1
        self.end_e = 0.1
        self.exploration_fraction = 0.5
        self.learning_starts = 5000
        self.train_frequency = 4

In [12]:
train(Args())

device_name: cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [32, 32, 20, 20]           2,080
              ReLU-2           [32, 32, 20, 20]               0
            Conv2d-3             [32, 64, 9, 9]          32,832
              ReLU-4             [32, 64, 9, 9]               0
            Conv2d-5             [32, 64, 7, 7]          36,928
              ReLU-6             [32, 64, 7, 7]               0
           Flatten-7                 [32, 3136]               0
           Dropout-8                 [32, 3136]               0
            Linear-9                  [32, 512]       1,606,144
          Dropout-10                  [32, 512]               0
             ReLU-11                  [32, 512]               0
           Linear-12                   [32, 12]           6,156
Total params: 1,684,140
Trainable params: 1,684,140
Non-trainable params: 0
----------

DependencyNotInstalled: Found neither the ffmpeg nor avconv executables. On OS X, you can install ffmpeg via `brew install ffmpeg`. On most Ubuntu variants, `sudo apt-get install ffmpeg` should do it. On Ubuntu 14.04, however, you'll need to install avconv with `sudo apt-get install libav-tools`. Alternatively, please install imageio-ffmpeg with `pip install imageio-ffmpeg`

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# !optuna-dashboard sqlite:///db.sqlite3